[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/evals/voyageai-embeddings-eval.ipynb)

[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/choose-embedding-model-rag/?utm_campaign=devrel&utm_source=cross-post&utm_medium=organic_social&utm_content=https%3A%2F%2Fgithub.com%2Fmongodb-developer%2FGenAI-Showcase&utm_term=apoorva.joshi)

# How to choose the right embedding model for your RAG application

This notebook evaluates the [voyage-3-large](https://docs.voyageai.com/embeddings/) model.


## Step 1: Install required libraries

- **datasets**: Python library to get access to datasets available on Hugging Face Hub
- **voyageai**: Python library to interact with Voyage AI's APIs
- **numpy**: Python library that provides tools to perform mathematical operations on arrays
- **pandas**: Python library for data analysis, exploration and manipulation
- **tdqm**: Python module to show a progress bar for loops


In [27]:
! pip install -qU datasets voyageai numpy pandas tqdm

## Step 2: Setup pre-requisites

Set the Voyage API key as an environment variable, and initialize the Voyage AI client.

Steps to obtain a Voyage AI API Key can be found [here](https://docs.voyageai.com/docs/api-key-and-installation#authentication-with-api-keys).


In [28]:
import getpass
import os

import voyageai

In [29]:
os.environ["VOYAGE_API_KEY"] = getpass.getpass("Voyage API Key:")
voyage_client = voyageai.Client()

Voyage API Key: ········


## Step 3: Download the evaluation dataset

We will use MongoDB's [cosmopedia-wikihow-chunked](https://huggingface.co/datasets/MongoDB/cosmopedia-wikihow-chunked) dataset, which has chunked versions of WikiHow articles from the [Cosmopedia](https://huggingface.co/datasets/HuggingFaceTB/cosmopedia) dataset released by Hugging Face. The dataset is pretty large, so we will only grab the first 2k records for testing.


In [30]:
import pandas as pd
from datasets import load_dataset

# Use streaming=True to load the dataset without downloading it fully
data = load_dataset("MongoDB/cosmopedia-wikihow-chunked", split="train", streaming=True)
# Get first 2k records from the dataset
data_head = data.take(2000)
df = pd.DataFrame(data_head)

# Use this if you want the full dataset
# data = load_dataset("AIatMongoDB/cosmopedia-wikihow-chunked", split="train")
# df = pd.DataFrame(data)

## Step 4: Data analysis

Make sure the length of the dataset is what we expect (2k), preview the data, drop Nones etc.


In [31]:
# Ensuring length of dataset is what we expect i.e. 2k
len(df)

2000

In [32]:
# Previewing the contents of the data
df.head()

,doc_id,chunk_id,text_token_length,text
0,0,0,180,Title: How to Create and Maintain a Compost Pi...
1,0,1,141,**Step 2: Gather Materials**\nGather brown (ca...
2,0,2,182,_Key guideline:_ For every volume of green mat...
3,0,3,188,_Key tip:_ Chop large items like branches and ...
4,0,4,157,**Step 7: Maturation and Use**\nAfter 3-4 mont...


In [33]:
# Only keep records where the text field is not null
df = df[df["text"].notna()]

In [34]:
# Number of unique documents in the dataset
df.doc_id.nunique()

352

## Step 5: Creating embeddings

Define the embedding function, and run a quick test.


In [35]:
from typing import List

In [36]:
def get_embeddings(text: str, input_type: str) -> List[float]:
    """
    Get embeddings using the Voyage AI API.

    Args:
        text (str): Text to embed
        input_type (str): Type of input to embed. Can be "document" or "query".

    Returns:
        List[float]: Embedding vector
    """
    response = voyage_client.embed(text, model="voyage-3-large", input_type=input_type)
    return response.embeddings[0]

In [37]:
# Generating a test embedding
test_voyageai_embed = get_embeddings(df.iloc[0]["text"], "document")

In [38]:
# Sanity check to make sure embedding dimensions are as expected i.e. 1024
len(test_voyageai_embed)

1024

## Step 6: Evaluation


### Measuring embedding latency

Create a local vector store (list) of embeddings for the entire dataset.


In [39]:
import numpy as np
from tqdm.auto import tqdm

In [40]:
texts = df["text"].tolist()

In [41]:
embeddings = []
# Generate embeddings
for text in tqdm(texts):
    embedding = get_embeddings(text, "document")
    # Add to the list of embeddings
    embeddings.append(np.array(embedding))

  0%|          | 0/2000 [00:00<?, ?it/s]

### Measuring retrieval quality

- Create embedding for the user query
- Get the top 5 most similar documents from the local vector store using cosine similarity as the similarity metric


In [18]:
from sentence_transformers.util import cos_sim

In [19]:
# Converting embeddings list to a Numpy array- required to calculate cosine similarity
embeddings = np.asarray(embeddings)

In [20]:
def query(query: str, top_k: int = 3) -> None:
    """
    Query the local vector store for the top 3 most relevant documents.

    Args:
        query (str): User query
        top_k (int, optional): Number of documents to return. Defaults to 3.
    """
    # Generate embedding for the user query
    query_emb = np.asarray(get_embeddings(query, "query"))
    # Calculate cosine similarity
    scores = cos_sim(query_emb, embeddings)[0]
    # Get indices of the top k records
    idxs = np.argsort(-scores)[:top_k]

    print(f"Query: {query}")
    for idx in idxs:
        print(f"Score: {scores[idx]:.4f}")
        print(texts[idx])
        print("--------")

In [21]:
query("Give me some tips to improve my mental health.")

Query: Give me some tips to improve my mental health.
Score: 0.7262
Step 2: Reach Out to Someone Trustworthy
Connect with someone who cares about you—a friend, family member, mental health professional, or support group. Talking openly about your struggles can lighten your emotional burden, making it easier to manage. Sharing your feelings may also lead to helpful suggestions and advice from others.

Guideline: Make sure to choose someone trustworthy who has shown empathy towards you before. If possible, reach out to more than one person to build a strong support network around you.

Step 3: Engage in Physical Activity
Exercise releases endorphins, which improve mood and reduce stress levels. Go for a walk, jog, bike ride, swim, or engage in any physical activity that suits your abilities. Exercising regularly can significantly impact overall wellbeing by reducing symptoms of depression and anxiety.

Key Tip: Start small - aim for just five minutes of exercise if needed, then gradually

In [22]:
query("Give me some tips for writing good code.")

Query: Give me some tips for writing good code.
Score: 0.6609
Step 6: Improve Code Quality
Strive for clean, readable, maintainable code. Adopt consistent naming conventions, indentation styles, and formatting rules. Utilize version control systems like Git to track changes and collaborate effectively. Leverage linters and static analyzers to enforce style guides automatically. Document your work using comments and dedicated documentation tools. High-quality code facilitates collaboration, promotes longevity, and simplifies troubleshooting.

Step 7: Embrace Best Practices
Follow established best practices relevant to your chosen language and domain. Examples include Object-Oriented Design Principles, SOLID principles, Test-Driven Development (TDD), Dependency Injection, Asynchronous Programming, etc. While seemingly overwhelming initially, integrating them gradually enhances design patterns, scalability, and extensibility. Consult authoritative blogs, books, and articles to stay update

In [23]:
query("How to create a basic webpage?")

Query: How to create a basic webpage?
Score: 0.5843
Title: How to Get a Free Website with Way2Webscape.com

Introduction:
Way2Webscape.com offers free websites for users who want to create their own online presence without any cost. This tutorial will guide you through the process of getting a free website using this service. By following these steps, you'll be able to create an attractive and functional website that meets your needs.

Step 1: Visit Way2Webscape.com
Open your web browser and navigate to <https://www.way2webscape.com/>. This will take you to the homepage of Way2Webscape.com.

Key tip: Make sure you have a stable internet connection while completing the following steps.

Step 2: Click on "Create Your Free Website" Button
On the top right corner of the homepage, click on the green button labeled "Create Your Free Website." This action will redirect you to the registration page.
--------
Score: 0.5713
Step 6: Choose a Template
After logging in, select a template that suits

In [24]:
query(
    "What are some environment-friendly practices I can incorporate in everyday life?"
)

Query: What are some environment-friendly practices I can incorporate in everyday life?
Score: 0.6947
a) Bring reusable bags, containers, utensils, and water bottles when shopping or dining out.
b) Purchase items with minimal packaging, prioritize products made from recycled materials, and avoid single-use plastics.
c) Participate in composting programs or start composting at home to turn organic waste into nutrient-rich soil amendments.
d) Donate unwanted items rather than discarding them, and repurpose household goods when possible.
e) Learn about local recycling rules and sort waste accordingly.

Key Tip: Familiarize yourself with your city's waste management system to optimize recycling efforts and divert waste from landfills.

Step 4: Protect Biodiversity
Biodiversity underpins the health and resilience of ecosystems, providing vital services such as pollination, decomposition, pest control, and nutrient cycling. Safeguard biodiversity through the following actions:
--------
Score